In [3]:
import os
import pandas as pd 

STATIC_PATH = '/home/elinha/NLP/moducon_data'
north_america_file_name = 'review_data_japan_china_american.csv'

north_america_path= os.path.join(STATIC_PATH, north_america_file_name)
na_df = pd.read_csv(north_america_path)

In [4]:
from sentence_transformers import SentenceTransformer, util
transformer_path = 'all-MiniLM-L6-v2'  # 5 times faster and still offers good quality
embedder = SentenceTransformer(transformer_path)


model.safetensors: 100%|██████████| 90.9M/90.9M [00:07<00:00, 11.5MB/s]
model.onnx: 100%|██████████| 90.4M/90.4M [00:08<00:00, 10.5MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:08<00:00, 10.9MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 12.6kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 156kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.11MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 665kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 17.1MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 13.0MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 665kB/s]


In [5]:
corpus = na_df['cleaned_sentence'].values.tolist()
corpus_embeddings = embedder.encode(corpus) 

In [6]:
from sklearn.cluster import KMeans
num_clusters = 20
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

cluster_dict = {}
for i, cluster in enumerate(clustered_sentences):
    cluster_dict[f"review_cluster_{i}"] = cluster

/home/elinha/.pyenv/versions/3.9.17/envs/nlp_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
for k, v in cluster_dict.items():
    print(k, len(v))

negavite_review_cluster_0 502
negavite_review_cluster_1 228
negavite_review_cluster_2 656
negavite_review_cluster_3 652
negavite_review_cluster_4 1197
negavite_review_cluster_5 443
negavite_review_cluster_6 823
negavite_review_cluster_7 576
negavite_review_cluster_8 366
negavite_review_cluster_9 651
negavite_review_cluster_10 611
negavite_review_cluster_11 564
negavite_review_cluster_12 479
negavite_review_cluster_13 416
negavite_review_cluster_14 354
negavite_review_cluster_15 504
negavite_review_cluster_16 1009
negavite_review_cluster_17 919
negavite_review_cluster_18 1144
negavite_review_cluster_19 478


In [15]:
len(cluster_dict.keys())

20

In [14]:
len(cluster_dict.values())

20

In [16]:
na_cluster_df = pd.DataFrame(
    {"cluster": cluster_dict.keys(), "sentences": cluster_dict.values()}
    )

In [18]:
cluster_df = pd.DataFrame()

for k,v in cluster_dict.items():
    df = pd.DataFrame({"review": v})
    df['cluster'] = k
    cluster_df = pd.concat([cluster_df, df])

In [19]:
cluster_df

,review,cluster
0,"It's so well written, the characters are all a...",negavite_review_cluster_0
1,desirous character is too good to be true.,negavite_review_cluster_0
2,If Jules' husband was played by some actor wit...,negavite_review_cluster_0
3,Both the stars acting compliments each other.,negavite_review_cluster_0
4,"The two stars are fantastic and well matched, ...",negavite_review_cluster_0
...,...,...
473,The rest of the cast was great too.,negavite_review_cluster_19
474,All the other main characters/actors were deli...,negavite_review_cluster_19
475,All around the actors who filled roles in this...,negavite_review_cluster_19
476,These shifts are executed very well through th...,negavite_review_cluster_19


In [21]:
cluster_df = cluster_df.drop_duplicates(subset=['review'])


In [22]:
cluster_df

,review,cluster
0,"It's so well written, the characters are all a...",negavite_review_cluster_0
1,desirous character is too good to be true.,negavite_review_cluster_0
2,If Jules' husband was played by some actor wit...,negavite_review_cluster_0
3,Both the stars acting compliments each other.,negavite_review_cluster_0
4,"The two stars are fantastic and well matched, ...",negavite_review_cluster_0
...,...,...
473,The rest of the cast was great too.,negavite_review_cluster_19
474,All the other main characters/actors were deli...,negavite_review_cluster_19
475,All around the actors who filled roles in this...,negavite_review_cluster_19
476,These shifts are executed very well through th...,negavite_review_cluster_19


In [26]:
template = """
              Write a summary of the following text delimited by triple backticks.
              Return your response which covers the key points of the text.
              ```{text}```
              SUMMARY:
           """


In [38]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [28]:
test_text_list = cluster_df[cluster_df['cluster'] == 'negavite_review_cluster_0']['review'].to_list()
test_text = " ".join(test_text_list)

In [30]:
len(test_text)

61717

In [47]:
cluster_df.to_csv(os.path.join(STATIC_PATH, 'cluster_df.csv'), index=False, encoding='utf-8-sig')

## 요약

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [45]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""


model.safetensors: 100%|██████████| 1.63G/1.63G [02:28<00:00, 10.9MB/s]
generation_config.json: 100%|██████████| 363/363 [00:00<00:00, 111kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 2.49MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 856kB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 1.48MB/s]


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [53]:
len(ARTICLE)

2298

In [54]:
cluster_0 = summarizer(test_text[:2298], max_length=130, min_length=30, do_sample=False)

In [55]:
cluster_0

[{'summary_text': 'Both the stars acting compliments each other. The only negative that I could grasp from this movie is the fact that all the younger people took a page from De tiro\'s "experience" and started molding towards his way of life. They aren\'t developed as characters, they\'re just onlookers to their successes.'}]

In [56]:

max_length = 1024  # LLM의 최대 길이 제한

# 텍스트를 토큰 단위로 나누기
tokens = tokenizer.encode(test_text)
num_tokens = len(tokens)

In [ ]:
print(llm_chain.run(test_text))

In [41]:
llm_chain_result= []

In [52]:
# 텍스트를 max_length 크기로 청크로 나누기
for i in range(0, num_tokens, max_length):
    chunk_tokens = tokens[i:i+max_length]
    input_ids = torch.tensor(chunk_tokens).unsqueeze(0)  # 배치 차원 추가

    with torch.no_grad():
        outputs = summarizer(test_text, max_length=130, min_length=30, do_sample=False)
    
    # generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_chain_result.append(outputs)

# 결과 출력
for idx, result in enumerate(llm_chain_result):
    print(f"Chunk {idx+1}:")
    print(result)
    print("\n" + "="*50 + "\n")

IndexError: index out of range in self

In [43]:
llm_chain_result

[' """\n            # Get the text\n            text = self.text\n            # Split text into paragraphs\n            paragraphs = text.split("\\n\\n")\n            # Get the summary of the text\n            summary = self.get_summary(paragraphs)\n            # Return the summary\n            return summary\n\n        # Define the function to get the summary of the text\n        def get_summary(paragraphs):\n            # Initialize the summary\n            summary = ""\n            # Iterate over the paragraphs\n            for paragraph in paragraphs:\n                # Check if the paragraph is not empty\n                if paragraph:\n                    # Split the paragraph into sentences\n                    sentences = paragraph.split(".")\n                    # Iterate over the sentences\n                    for sentence in sentences:\n                        # Check if the sentence is not empty\n                        if sentence:\n                            # Add the sen